# The Battle of Neighborhoods
## Which neighborhood is most suitable for opening a coworking space in the Netherlands?

In [ ]:
### 1. Introduction

#### 1. Scraping the list of postal codes in Overijssel

In [3]:
# import modules to be used for this assignment

import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [10]:
# The URL offering the list of postal codes in Overijssel

url_O = "https://www.metatopos.eu/overijssel2.html"

In [14]:
# Read HTML tables into a list of DataFrame objects

df = pd.read_html(url_O)

In [18]:
df

[                                                     0  \
 0    Gemeente(in de provincieOverijssel)met link na...   
 1                                             Deventer   
 2                                             Deventer   
 3                                             Deventer   
 4                                             Deventer   
 ..                                                 ...   
 542                                    Steenwijkerland   
 543                                    Steenwijkerland   
 544                                    Steenwijkerland   
 545                                    Steenwijkerland   
 546                                    Steenwijkerland   
 
                                                      1  \
 0    Naam woonplaatszoals ingeschrevenbij het Kadas...   
 1                                             Deventer   
 2                                             Deventer   
 3                                             Devente

In [25]:
# Define the table in the website as the Overijssel dataframe

df_overijssel = df[0]

In [26]:
df_overijssel

,0,1,2,3,4
0,Gemeente(in de provincieOverijssel)met link na...,Naam woonplaatszoals ingeschrevenbij het Kadas...,Plaatsen enbuurtschappenzoals vermeldbij de be...,Post-code,Wijknaamof omschrijvingof ligging van de wijk ...
1,Deventer,Deventer,Deventer,7411,"centrum, Knutteloord"
2,Deventer,Deventer,Deventer,7412,"Zandweerd, Zwolse Wijk"
3,Deventer,Deventer,Deventer,7413,Voorstad
4,Deventer,Deventer,Deventer,7414,"Platvoet, Borgele"
...,...,...,...,...,...
542,Steenwijkerland,Oldemarkt,Oldemarkt,8375,NaN
543,Steenwijkerland,Oldemarkt,De Hare,8375,NaN
544,Steenwijkerland,Ossenzijl,Ossenzijl,8376,NaN
545,Steenwijkerland,Kalenberg,Kalenberg,8377,NaN


In [30]:
# Drop unneccesary columns

df_overijssel.drop([1, 2], axis = 1, inplace = True)

In [34]:
# Rename the columns

df_overijssel.rename(columns = {0:"City", 3:"Postal Code", 4:"Neighbourhood"}, inplace = True)

In [35]:
# Drop the first row and change the sequence of columns

df_overijssel.drop([0], inplace = True)
df_overijssel = df_overijssel[['Postal Code', 'City', 'Neighbourhood']]

In [36]:
df_overijssel

,Postal Code,City,Neighbourhood
1,7411,Deventer,"centrum, Knutteloord"
2,7412,Deventer,"Zandweerd, Zwolse Wijk"
3,7413,Deventer,Voorstad
4,7414,Deventer,"Platvoet, Borgele"
5,7415,Deventer,Keizerslanden
...,...,...,...
542,8375,Steenwijkerland,NaN
543,8375,Steenwijkerland,NaN
544,8376,Steenwijkerland,NaN
545,8377,Steenwijkerland,NaN


In [37]:
# Drop the rows whose values are 'NaN'

df_overijssel.dropna(inplace = True)

#### 2. Acquiring the latitude and longitutde of each postal code using Bing Map API

In [42]:
# Import geocoder

import geocoder

In [44]:
# Add two columns 'Latitude' and 'Longitude'

df_overijssel['Latitude'] = ''
df_overijssel['Longitude'] = ''

df_overijssel.head()

,Postal Code,City,Neighbourhood,Latitude,Longitude
1,7411,Deventer,"centrum, Knutteloord",,
2,7412,Deventer,"Zandweerd, Zwolse Wijk",,
3,7413,Deventer,Voorstad,,
4,7414,Deventer,"Platvoet, Borgele",,
5,7415,Deventer,Keizerslanden,,


In [46]:
# Fill up the two columns using loop

for i in range(len(df_overijssel)):
    g = geocoder.bing('{}, the Netherlands, Overijssel, {}'.format(df_overijssel.iloc[i, 0], df_overijssel.iloc[i, 1]), key = "AveqYCAVfTS2P8hy16Zdv1JCwo_5CbSiObipo9UsshRtze28C--AYls_MLl0eFLd")
    df_overijssel.iloc[i, 3] = g.latlng[0]
    df_overijssel.iloc[i, 4] = g.latlng[1]

In [47]:
df_overijssel

,Postal Code,City,Neighbourhood,Latitude,Longitude
1,7411,Deventer,"centrum, Knutteloord",52.255886,6.16137
2,7412,Deventer,"Zandweerd, Zwolse Wijk",52.26442,6.135583
3,7413,Deventer,Voorstad,52.26189,6.158082
4,7414,Deventer,"Platvoet, Borgele",52.274899,6.151459
5,7415,Deventer,Keizerslanden,52.270542,6.165892
...,...,...,...,...,...
495,8293,Kampen,Mastenbroek-west (=ten westen van de Oude Wete...,52.561821,5.99851
498,8294,Zwartewaterland,Mastenbroek-oost (=ten oosten van de Oude Wete...,52.573589,6.022351
508,8331,Steenwijkerland,Steenwijk-centrum en -zuid: (ten zuiden van de...,52.786621,6.116048
509,8332,Steenwijkerland,Steenwijk-noord: (ten noorden van de spoorlijn...,52.798843,6.13266


In [48]:
# Reset the index

df_overijssel.reset_index(drop = True, inplace = True)

In [49]:
df_overijssel

,Postal Code,City,Neighbourhood,Latitude,Longitude
0,7411,Deventer,"centrum, Knutteloord",52.255886,6.16137
1,7412,Deventer,"Zandweerd, Zwolse Wijk",52.26442,6.135583
2,7413,Deventer,Voorstad,52.26189,6.158082
3,7414,Deventer,"Platvoet, Borgele",52.274899,6.151459
4,7415,Deventer,Keizerslanden,52.270542,6.165892
...,...,...,...,...,...
133,8293,Kampen,Mastenbroek-west (=ten westen van de Oude Wete...,52.561821,5.99851
134,8294,Zwartewaterland,Mastenbroek-oost (=ten oosten van de Oude Wete...,52.573589,6.022351
135,8331,Steenwijkerland,Steenwijk-centrum en -zuid: (ten zuiden van de...,52.786621,6.116048
136,8332,Steenwijkerland,Steenwijk-noord: (ten noorden van de spoorlijn...,52.798843,6.13266


#### 3. Exploring and clustering the neighbourhoods in Overijssel

In [50]:
# import libraries to be used

import matplotlib.cm as cm
import matplotlib.colors as colors
import folium

In [51]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

# import Nominatim converting an address into latitude and longitude values
from geopy.geocoders import Nominatim

In [53]:
# Use geopy library to get the latitude and longitude values of Overijssel, the Netherlands

address = 'Overijssel, the Netherlands'

geolocator = Nominatim(user_agent = "Overijssel Explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

print("The geographical coordinate of Overijssel are {}, {}.".format(latitude, longitude))

The geographical coordinate of Overijssel are 52.4254143, 6.4610611.


In [55]:
# Create a map of Overijssel with neighbourhoods superimposed on top

map_overijssel = folium.Map(location = [latitude, longitude], zoom_start = 10)

# Add markers to map

for lat, lng, neighbourhood, city in zip(df_overijssel["Latitude"], df_overijssel["Longitude"], df_overijssel["Neighbourhood"], df_overijssel["City"]):
    label = '{}, {}'.format(neighbourhood, city)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker([lat, lng], radius = 5, popup = label, color = 'blue', fill = True, fill_color = "#3186cc", fill_opacity = 0.7, parse_html = False).add_to(map_overijssel)

map_overijssel

In [56]:
# Define Foursquare Credentials and Version

CLIENT_ID = "H1RCC0M44AYUJUXSK4H3IBJCEAGRS54WJHD3ABHKNUBNGAI0"
CLIENT_SECRET = '4DGTG1NSFOP1KU333FRGDPRHAWB0FW2YC523ZPDKIG1D042B' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: H1RCC0M44AYUJUXSK4H3IBJCEAGRS54WJHD3ABHKNUBNGAI0
CLIENT_SECRET:4DGTG1NSFOP1KU333FRGDPRHAWB0FW2YC523ZPDKIG1D042B
